In [1]:
from dotenv import load_dotenv
import json
load_dotenv()


from helpers.wttn_agent import get_attn_agent, WttnFullReport
from helpers.model_client import get_github_model_provider, get_openai_client
from helpers.trace_util import get_trace_url
from helpers.vis_workflow import vis_agent


from agents import(
    Runner,
    RunConfig,

    trace,
    ToolCallOutputItem,

)


In [2]:
agent = get_attn_agent()

In [3]:
vis_agent(agent, "Weather_Workflow")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


'viz/Weather_Workflow.gv'

In [4]:
agent.tools

[FunctionTool(name='weather_data_collector', description='you collect data about weather from different data sources for a given city or location', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'weather_data_collector_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000024AD8862200>, strict_json_schema=True)]

In [5]:
agent.handoffs

[Agent(name='whether_report_summariser_agent', instructions='\n        you can read a detailed structured weather report, and then write a weather report summary for the user.\n        the summary can be ready by a human and should be easy to understand.\n        the report should include information about the weather conditions now,\n        summary of changes in the weather over the next three days,\n        with special attention to any extreme changes during the day or over the next few days\n        ', handoff_description='You are writing a weather report summary for the user from structured and detailed weather report data', handoffs=[], model='o3-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=16000, reasoning=None, metadata=None, store=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=<class

In [6]:
city = input("Enter the city name: ") or "Sydney"

with trace(workflow_name="wttn_agent", group_id="wttn_agent") as tr:
    print(get_trace_url(tr))
    result = await Runner.run(
        starting_agent=agent,
        input=f"what is the weather today in {city}",
        run_config=RunConfig(
            model_provider=get_github_model_provider(get_openai_client()),
        )
    )

    print(result.final_output)


https://platform.openai.com/traces/trace?trace_id=trace_bbb0b7552dc0474296a6d00e79d2cd71
[debug] get_wttn called with city: Perth
Here's a summary of the weather in Perth:

**Today (Wednesday):** The day is currently sunny with a temperature of 16°C and light winds from the west at 11 km/h. Moving forward, the weather remains sunny throughout the day, with the temperature peaking at 25°C by noon, before cooling slightly to 22°C in the evening. Wind speeds will increase to 22-32 km/h in the evening. Late at night, expect partly cloudy skies with winds at 19-31 km/h.

**Thursday:** Another sunny day with clear skies expected. Temperatures will start at 20°C in the morning and rise to a high of 25°C by noon. However, stronger winds are forecasted in the evening, reaching speeds of 37-50 km/h, with cooler temperatures around 21°C. Clear night skies with winds of 31-41 km/h are anticipated.

**Friday:** The day begins sunny at 17°C and reaches 21°C by noon, with moderate to strong breezes t

In [9]:
result.raw_responses[-1].usage

Usage(requests=1, input_tokens=2466, output_tokens=274, total_tokens=2740)

In [11]:
from IPython.display import display, HTML, Markdown
display(Markdown(result.final_output))


Here's a summary of the weather in Perth:

**Today (Wednesday):** The day is currently sunny with a temperature of 16°C and light winds from the west at 11 km/h. Moving forward, the weather remains sunny throughout the day, with the temperature peaking at 25°C by noon, before cooling slightly to 22°C in the evening. Wind speeds will increase to 22-32 km/h in the evening. Late at night, expect partly cloudy skies with winds at 19-31 km/h.

**Thursday:** Another sunny day with clear skies expected. Temperatures will start at 20°C in the morning and rise to a high of 25°C by noon. However, stronger winds are forecasted in the evening, reaching speeds of 37-50 km/h, with cooler temperatures around 21°C. Clear night skies with winds of 31-41 km/h are anticipated.

**Friday:** The day begins sunny at 17°C and reaches 21°C by noon, with moderate to strong breezes throughout the day (winds at 21-31 km/h). Clear skies persist into the evening and night, cooling to 16°C by nighttime.

**Key Highlights:** The weather remains dry and sunny throughout, with no rain expected for the next three days. Be prepared for an increase in wind intensity on Thursday evening.

In [ ]:
for r in result.raw_responses:
    print(f"Usage: {r.usage.total_tokens}, Output: {r.output}")



In [ ]:
tool_call_output = None
for item in result.new_items:
    print(item)
    if isinstance(item, ToolCallOutputItem):
        tool_call_output = item
        # print(item.output)
        # print((json.loads(item.output)))
        item_output : WttnFullReport = item.output
        print(item_output)


In [ ]:
api_output = json.loads(tool_call_output.raw_item["output"])

In [ ]:
print(api_output['weather_wttn_raw_response'])